In [76]:
import os
os.chdir("../../")
import numpy as np
from pandas import DataFrame
from backtesting import Backtest
from backtesting.lib import Strategy

from backtesting.test import EURUSD
from datetime import datetime

In [77]:
class OcoBacktestStrategy(Strategy):
    def init(self):
        self.position_open = False
        self.data.df.reset_index(inplace=True, names="Datetime")
        self.sl_pips = 200

    def get_current_hour(self):
        date_string = self.data.Datetime[-1]
        date_object = np.datetime_as_string(date_string)
        date_object = date_object[:-5]
        date_object = datetime.strptime(date_object, "%Y-%m-%dT%H:%M:%S.%f")
        hour = date_object.hour

        return hour
    
    def calculate_new_price(
        self, original_price, pip_value, pip_factor=0.0001, add_pips=True
    ):
        direction = 1 if add_pips else -1
        new_price = original_price + direction * pip_value * pip_factor
        return new_price

    def calculate_pips_offset(self, price):
        pips_offset = 10
        buy_price = self.calculate_new_price(price, pips_offset)
        sell_price = self.calculate_new_price(price, pips_offset, add_pips=False)
        return buy_price, sell_price

    def next(self):
        current_hour = self.get_current_hour()
        buy_price, sell_price = self.calculate_pips_offset(self.data.Close[-1])

        buy_tp = self.calculate_new_price(buy_price, 10)
        buy_sl = self.calculate_new_price(buy_price, 50, add_pips=False)
        sell_tp = self.calculate_new_price(sell_price, 10, add_pips=False)
        sell_sl = self.calculate_new_price(sell_price, 50)

        if not self.position or len(self.orders) == 0:
            if current_hour == 0:
            # Check if there are no open positions
            
                # Place buy and sell limit orders
                buy_order = self.buy(limit=buy_price, tp=buy_tp, sl=buy_sl)
                sell_order = self.sell(limit=sell_price, tp=sell_tp, sl=sell_sl)
                print(buy_order, sell_order)
                print(self.orders)
                print("-"*100)
        else:
            print(self.orders)
            if len(self.orders) > 0:
                for order in self.orders:
                    print(self.position, order)
                    if (self.position.size == order.size):
                        print(self.position.size)
                        print(order.size)

In [78]:
bt = Backtest(EURUSD, OcoBacktestStrategy,
              cash=10000, commission=.002)

output = bt.run()
# bt.plot()

<Order size=1.0, limit=1.07264, sl=1.06764, tp=1.07364, contingent=0> <Order size=-1.0, limit=1.07064, sl=1.07564, tp=1.06964, contingent=0>
(<Order size=1.0, limit=1.07264, sl=1.06764, tp=1.07364, contingent=0>, <Order size=-1.0, limit=1.07064, sl=1.07564, tp=1.06964, contingent=0>)
----------------------------------------------------------------------------------------------------
(<Order size=-9313.0, stop=1.06764, contingent=1>, <Order size=-9313.0, limit=1.07364, contingent=1>)
<Position: 9313 (1 trades)> <Order size=-9313.0, stop=1.06764, contingent=1>
<Position: 9313 (1 trades)> <Order size=-9313.0, limit=1.07364, contingent=1>
(<Order size=-9313.0, stop=1.06764, contingent=1>, <Order size=-9313.0, limit=1.07364, contingent=1>)
<Position: 9313 (1 trades)> <Order size=-9313.0, stop=1.06764, contingent=1>
<Position: 9313 (1 trades)> <Order size=-9313.0, limit=1.07364, contingent=1>
(<Order size=-9313.0, stop=1.06764, contingent=1>, <Order size=-9313.0, limit=1.07364, contingent=1>

/Users/pimpmasterguapo/Documents/github/personal_projects/ai-trading-app/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:912: UserWarning: (736) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
/Users/pimpmasterguapo/Documents/github/personal_projects/ai-trading-app/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:912: UserWarning: (1216) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubi

<Order size=1.0, limit=1.14256, sl=1.13756, tp=1.14356, contingent=0> <Order size=-1.0, limit=1.14056, sl=1.14556, tp=1.13956, contingent=0>
(<Order size=1.0, limit=1.14256, sl=1.13756, tp=1.14356, contingent=0>, <Order size=-1.0, limit=1.14056, sl=1.14556, tp=1.13956, contingent=0>)
----------------------------------------------------------------------------------------------------
(<Order size=-8159.0, stop=1.13756, contingent=1>, <Order size=-8159.0, limit=1.14356, contingent=1>)
<Position: 8159 (1 trades)> <Order size=-8159.0, stop=1.13756, contingent=1>
<Position: 8159 (1 trades)> <Order size=-8159.0, limit=1.14356, contingent=1>
(<Order size=-8159.0, stop=1.13756, contingent=1>, <Order size=-8159.0, limit=1.14356, contingent=1>)
<Position: 8159 (1 trades)> <Order size=-8159.0, stop=1.13756, contingent=1>
<Position: 8159 (1 trades)> <Order size=-8159.0, limit=1.14356, contingent=1>
(<Order size=-8159.0, stop=1.13756, contingent=1>, <Order size=-8159.0, limit=1.14356, contingent=1>

/Users/pimpmasterguapo/Documents/github/personal_projects/ai-trading-app/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:912: UserWarning: (1528) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
/Users/pimpmasterguapo/Documents/github/personal_projects/ai-trading-app/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:912: UserWarning: (1888) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dub

(<Order size=-7323.0, stop=1.189, contingent=1>, <Order size=-7323.0, limit=1.195, contingent=1>)
<Position: 7323 (1 trades)> <Order size=-7323.0, stop=1.189, contingent=1>
<Position: 7323 (1 trades)> <Order size=-7323.0, limit=1.195, contingent=1>
<Order size=1.0, limit=1.19749, sl=1.19249, tp=1.19849, contingent=0> <Order size=-1.0, limit=1.19549, sl=1.20049, tp=1.19449, contingent=0>
(<Order size=1.0, limit=1.19749, sl=1.19249, tp=1.19849, contingent=0>, <Order size=-1.0, limit=1.19549, sl=1.20049, tp=1.19449, contingent=0>)
----------------------------------------------------------------------------------------------------
(<Order size=-7300.0, stop=1.19249, contingent=1>, <Order size=-7300.0, limit=1.19849, contingent=1>)
<Position: 7300 (1 trades)> <Order size=-7300.0, stop=1.19249, contingent=1>
<Position: 7300 (1 trades)> <Order size=-7300.0, limit=1.19849, contingent=1>
(<Order size=-7300.0, stop=1.19249, contingent=1>, <Order size=-7300.0, limit=1.19849, contingent=1>)
<Posit

/Users/pimpmasterguapo/Documents/github/personal_projects/ai-trading-app/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:912: UserWarning: (2632) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
/Users/pimpmasterguapo/Documents/github/personal_projects/ai-trading-app/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:912: UserWarning: (3378) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dub

<Order size=1.0, limit=1.18811, sl=1.18311, tp=1.18911, contingent=0> <Order size=-1.0, limit=1.18611, sl=1.19111, tp=1.18511, contingent=0>
(<Order size=1.0, limit=1.18811, sl=1.18311, tp=1.18911, contingent=0>, <Order size=-1.0, limit=1.18611, sl=1.19111, tp=1.18511, contingent=0>)
----------------------------------------------------------------------------------------------------
(<Order size=-6565.0, stop=1.18311, contingent=1>, <Order size=-6565.0, limit=1.18911, contingent=1>)
<Position: 6565 (1 trades)> <Order size=-6565.0, stop=1.18311, contingent=1>
<Position: 6565 (1 trades)> <Order size=-6565.0, limit=1.18911, contingent=1>
(<Order size=-6565.0, stop=1.18311, contingent=1>, <Order size=-6565.0, limit=1.18911, contingent=1>)
<Position: 6565 (1 trades)> <Order size=-6565.0, stop=1.18311, contingent=1>
<Position: 6565 (1 trades)> <Order size=-6565.0, limit=1.18911, contingent=1>
(<Order size=-6565.0, stop=1.18311, contingent=1>, <Order size=-6565.0, limit=1.18911, contingent=1>

/Users/pimpmasterguapo/Documents/github/personal_projects/ai-trading-app/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:912: UserWarning: (4793) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(


In [79]:
output

Start                     2017-04-19 09:00:00
End                       2018-02-07 15:00:00
Duration                    294 days 06:00:00
Exposure Time [%]                       42.74
Equity Final [$]                  7144.632221
Equity Peak [$]                       10000.0
Return [%]                         -28.553678
Buy & Hold Return [%]               14.628937
Return (Ann.) [%]                  -28.649319
Volatility (Ann.) [%]                 2.47183
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -28.798164
Avg. Drawdown [%]                  -28.798164
Max. Drawdown Duration      293 days 15:00:00
Avg. Drawdown Duration      293 days 15:00:00
# Trades                                  188
Win Rate [%]                          1.06383
Best Trade [%]                       0.181854
Worst Trade [%]                     -0.566823
Avg. Trade [%]                    

In [80]:
output["_trades"]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,9313,16,21,1.073743,1.07364,-0.961102,-0.000096,16,21,5
1,9311,40,46,1.073863,1.07378,-0.776910,-0.000078,40,46,6
2,9196,64,68,1.087230,1.08704,-1.748344,-0.000175,64,68,4
3,9184,88,93,1.088433,1.08824,-1.768104,-0.000177,88,93,5
4,9119,112,119,1.095948,1.08979,-56.150425,-0.005618,112,119,7
...,...,...,...,...,...,...,...,...,...,...
183,5806,4889,4897,1.244584,1.24410,-2.811265,-0.000389,4889,4897,8
184,5764,4913,4925,1.253161,1.24672,-37.127768,-0.005140,4913,4925,12
185,5757,4937,4950,1.248272,1.24181,-37.199201,-0.005176,4937,4950,13
186,5762,4961,4967,1.240777,1.24031,-2.688549,-0.000376,4961,4967,6
